In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.pardir, os.pardir, 'src'))

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import fmin
import seaborn as sns
import statsmodels.api as sm

from data_processing.rat import load_rat_items
from data_processing.generate_association_matrix import load_assoc_mat
from data_processing.spgen import load_pointers
from model.stimulus import filter_valid
from data_processing.spgen import load_pointers

%matplotlib inline

## Load experimental data

In [2]:
basedir = os.path.join(os.pardir, os.pardir)
path_file = os.path.join(basedir, 'data', 'raw', '144CompoundBowden.xlsx')
xls = pd.ExcelFile(path_file)

Load the excel sheet with values from the paper and sort RAT problems according to the t=2 condition:

In [3]:
df = xls.parse('RAT stimuli part1.csv', skip_footer=2)
df.columns = [
    'items', 'solution', '2s-%',
    '7s-%',  '7s-t-mean', '7s-t-sd',
    '15s-%', '15s-t-mean', '15s-t-sd',
    '30s-%', '30s-t-mean', '30s-t-sd']
df = df.sort_values('2s-%', ascending=False)
df = df.set_index(np.arange(len(df)))

## Load association data

In [4]:
assoc, i2w, w2i = load_assoc_mat(os.path.join(basedir, 'data', 'associationmatrices'), 'freeassoc_asymmetric')

In [6]:
pointers, i2w, w2i = load_pointers(
    os.path.join(basedir, 'data', 'semanticpointers'),
    'freeassoc_asymmetric_randomize_5018w_512d')
assoc = datasets['freeassoc_asymmetric'][0]
pointers_sq = np.dot(pointers, pointers.T)
datasets['neural'] = (np.dot(pointers_sq, np.dot(assoc, pointers_sq)), i2w, w2i)

In [13]:
def solveable(row, assoc, i2w, w2i):
    cues = row['items'].upper().split('/')
    target = row['solution'].upper()
    
    if not all(w in i2w for w in cues+[target]):
        return np.nan
    else:
        strengths = np.sum([assoc[w2i[cue], :] for i, cue in enumerate(cues)], axis=0)
        return strengths[w2i[target]] > 0.

In [15]:
solvable_problems = df.apply(solveable, args=(assoc, i2w, w2i), axis=1)

In [16]:
len(solvable_problems.dropna()) - solvable_problems.sum()

37